# 01) INSTALAÇÃO DA BIBLIOTECA

In [2]:
pip install opencv-python matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# 02) CARREGAR IMAGENS

In [3]:
import cv2
import glob
import os
import numpy as np
from pathlib import Path

# Caminhos relativos (corretos para quem está na pasta notebooks/)
path_raw = Path("../data/raw/")
images = [cv2.imread(str(file)) for file in path_raw.glob("*.jpg")]

# Verifique se as imagens foram carregadas corretamente
print(f"Total de imagens carregadas: {len(images)}")

Total de imagens carregadas: 197


##### Carregamento de imagens

As imagens foram carregadas diretamente da pasta `data/raw/`. Essas imagens representam cenas urbanas com possíveis grafites. Serão usadas como base para todas as etapas de pré-processamento e detecção.


## 03) Conversão para escala de cinza

In [4]:
gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]

##### Conversão para escala de cinza

Converter as imagens para tons de cinza reduz a complexidade dos dados, mantendo apenas a informação de intensidade luminosa. Essa etapa é essencial antes de aplicar filtros e detectar bordas, pois evita processamentos desnecessários nas cores.


## Aplicação dos filtros espaciais

### 04) Filtro Gaussiano para suavizar a imagem e reduzir ruído:

In [5]:
gaussian_images = [cv2.GaussianBlur(img, (5,5), 0) for img in gray_images]

##### Filtro Gaussiano

O filtro Gaussiano aplica um desfoque suave na imagem para reduzir ruídos e detalhes muito pequenos. Isso ajuda a evitar que o detector de bordas capture regiões irrelevantes. Ele é indicado quando há ruído do tipo "granulação" na imagem.


### 05) Filtro Mediano

In [6]:
median_images = [cv2.medianBlur(img, 5) for img in gray_images]

##### Filtro Mediano

O filtro Mediano também reduz ruídos, mas de maneira diferente do Gaussiano. Ele preserva melhor as bordas e remove pontuais isolados (ruído "sal e pimenta"). Ideal para cenas urbanas com muitos contrastes e detalhes finos.


## Detecção de bordas

### 06) Canny

In [7]:
canny_edges = [cv2.Canny(img, threshold1=100, threshold2=200) for img in gaussian_images]

##### Detecção de bordas com Canny

O algoritmo de Canny identifica bordas com base em transições abruptas de intensidade. Utiliza dois limiares (baixo e alto) para decidir quais bordas manter. É amplamente usado por sua eficiência e precisão.


### 07) Sobel

In [8]:
sobel_edges = [cv2.Sobel(img, cv2.CV_64F, 1, 1, ksize=3) for img in gaussian_images]

##### Detecção de bordas com Sobel

O filtro de Sobel calcula o gradiente da imagem nas direções horizontal e vertical. Ele detecta bordas com orientação e é útil para estruturas como traços de grafites, que muitas vezes possuem linhas retas.


## 08) Salvar resultados processados

In [9]:
from pathlib import Path

processed_path = Path("../data/processed/")
results_path = Path("../results/")

processed_path.mkdir(parents=True, exist_ok=True)
results_path.mkdir(parents=True, exist_ok=True)

for idx in range(len(images)):
    cv2.imwrite(str(processed_path / f"gray_{idx}.jpg"), gray_images[idx])
    cv2.imwrite(str(processed_path / f"gaussian_{idx}.jpg"), gaussian_images[idx])
    cv2.imwrite(str(processed_path / f"median_{idx}.jpg"), median_images[idx])
    cv2.imwrite(str(processed_path / f"canny_{idx}.jpg"), canny_edges[idx])
    cv2.imwrite(str(processed_path / f"sobel_{idx}.jpg"), np.uint8(np.absolute(sobel_edges[idx])))

## 09) Visualizar comparações

In [ ]:
import matplotlib.pyplot as plt

for i in range(len(images)):
    plt.figure(figsize=(12, 4))

    plt.subplot(131)
    plt.title('Original')
    plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))

    plt.subplot(132)
    plt.title('Gauss + Canny')
    plt.imshow(canny_edges[i], cmap='gray')

    plt.subplot(133)
    plt.title('Sobel')
    plt.imshow(np.uint8(np.absolute(sobel_edges[i])), cmap='gray')

    plt.tight_layout()
    plt.show()

##### Comparação visual dos filtros e bordas

Nesta etapa foram comparados os efeitos dos diferentes filtros (Gaussiano, Mediano) e detectores de borda (Canny, Sobel). Essa análise ajuda a entender qual combinação oferece melhor definição das áreas com grafite.


## 10) Detecção de contornos para realçar grafites

In [11]:
contoured_images = []

for i in range(len(canny_edges)):
    contoured = images[i].copy()
    contours, _ = cv2.findContours(canny_edges[i], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(contoured, contours, -1, (0, 255, 0), 2)
    contoured_images.append(contoured)

    # Salva a imagem com contornos
    cv2.imwrite(str(results_path / f"contornos_{i}.jpg"), contoured)

##### Realce das áreas com grafite (contornos)

As regiões detectadas pelo Canny foram contornadas nas imagens originais com a função `cv2.drawContours`. Isso permite visualizar claramente as áreas que o algoritmo considera como possíveis grafites, destacando-as em verde.


## 11) Extração de características de cor e textura

In [12]:
caracteristicas = []

for i in range(len(images)):
    img_rgb = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)
    mean_color = cv2.mean(img_rgb)[:3]  # média das 3 cores
    std_dev_texture = np.std(gray_images[i])  # desvio padrão como textura

    caracteristicas.append({
        "Imagem": f"imagem_{i}.jpg",
        "Média_R": mean_color[0],
        "Média_G": mean_color[1],
        "Média_B": mean_color[2],
        "Textura": std_dev_texture
    })

import pandas as pd
df_caracteristicas = pd.DataFrame(caracteristicas)
df_caracteristicas

,Imagem,Média_R,Média_G,Média_B,Textura
0,imagem_0.jpg,154.902487,115.257344,102.020313,24.869631
1,imagem_1.jpg,180.250898,183.176107,183.080508,27.825017
2,imagem_2.jpg,181.081914,176.919844,171.944258,29.063495
3,imagem_3.jpg,112.236315,90.088307,83.272565,55.698436
4,imagem_4.jpg,175.724336,151.857682,124.518177,56.233483
...,...,...,...,...,...
192,imagem_192.jpg,174.964727,169.318802,168.042773,68.331717
193,imagem_193.jpg,188.894818,167.517344,174.295065,56.840443
194,imagem_194.jpg,172.838724,178.169232,143.625182,54.110312
195,imagem_195.jpg,200.326940,191.925885,164.130924,41.477034


##### Extração de características (cor e textura)

Para futuras classificações automáticas, foram extraídas características básicas:
- **Cor média** (valores de R, G, B da imagem)
- **Textura** (desvio padrão da escala de cinza, que indica variações na imagem)

Esses dados foram armazenados em um arquivo `.csv` e podem ser usados para treinar modelos de IA no futuro.


## 12) Exportar características para CSV

In [13]:
df_caracteristicas.to_csv(str(results_path / "caracteristicas.csv"), index=False)